In [4]:
import sys
import os
src_dir = os.path.dirname(os.getcwd())
if src_dir not in sys.path:
    sys.path.append(src_dir)

import wandb
import torch
from src.models.baselines import InputIndependentBaselineModel, BaselineBERTLogisticRegressionModel
from src.models.lightning import LightingModelWrapper
from lightning import seed_everything
from src.data.data_loader import RottenDataLoader
from lightning import Trainer
from config import config
from lightning.pytorch.loggers import WandbLogger
from utils.wandb import wandb_login_ensure_personal_account
from training.callbacks import model_checkpoint_callback

from lightning.pytorch import callbacks
from pathlib import Path
from lightning.pytorch.callbacks.early_stopping import EarlyStopping

In [2]:
seed_everything(42)

torch.set_float32_matmul_precision('high')

Seed set to 42


In [3]:
wandb_login_ensure_personal_account()

wandb: Currently logged in as: szafrixxx. Use `wandb login --relogin` to force relogin


In [6]:
data = RottenDataLoader(config)
data.setup()


bert = BaselineBERTLogisticRegressionModel(config)
bert_l = LightingModelWrapper(bert)

wandb_logger = WandbLogger(project="rotten-tomatoes",
                            tags=[bert.__name__()],
                            log_model=True
                            )

wandb_logger.watch(bert_l)


trainer = Trainer(
    max_epochs=10,
    accelerator="auto",
    devices=1 if torch.cuda.is_available() else None,
    logger=wandb_logger,
    check_val_every_n_epoch=1,
    log_every_n_steps=1,
    callbacks=[model_checkpoint_callback()],
    detect_anomaly=False
)
trainer.fit(bert_l, datamodule=data)
wandb.finish()

wandb: logging graph, to disable use `wandb.watch(log_graph=False)`
You have turned on `Trainer(detect_anomaly=True)`. This will significantly slow down compute speed and is recommended only for model debugging.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
Saving the dataset (1/1 shards): 100%|██████████| 1066/1066 [00:00<00:00, 76953.08 examples/s]
/home/bszafranski/projects/private/rotten_tomatoes/.venv/lib/python3.11/site-packages/lightning/pytorch/callbacks/model_checkpoint.py:639: Checkpoint directory /home/bszafranski/projects/private/rotten_tomatoes/training/model_ckpts exists and is not empty.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name          | Type                                | Params
----------------------------------------------------------------------
0 | model         | BaselineBERTLogisticRegressionModel | 109 M 
1 | train_metrics | ModelMetrics        

Epoch 9: 100%|██████████| 67/67 [00:07<00:00,  8.95it/s, v_num=phzp, train/loss=0.360, val/loss=0.400]

`Trainer.fit` stopped: `max_epochs=10` reached.


Epoch 9: 100%|██████████| 67/67 [00:07<00:00,  8.77it/s, v_num=phzp, train/loss=0.360, val/loss=0.400]


wandb: WARNING Source type is set to 'repo' but some required information is missing from the environment. A job will not be created from this run. See https://docs.wandb.ai/guides/launch/create-job


epoch,▁▁▁▁▂▂▂▂▃▃▃▃▃▃▃▃▄▄▄▄▅▅▅▅▆▆▆▆▆▆▆▆▇▇▇▇████
train/accuracy,▁▇▆▇▇█▇▇▇▆
train/auroc,▁▇▇▇██▇▇▇▇
train/f1,▁▇▆▇▇█▇▇█▆
train/loss,▅▄▄▆▃▃▂▅▅▅▃▂▆▅▃▅▄▄▄▄▄▆▄▃▃▆▁▇▃█▅▃▄▃▃▅▂▃▄▃
train/precision,▁▇▆▇▇█▇▆▇▆
train/recall,▁▇▆▇███▇█▆
trainer/global_step,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
val/accuracy,▇▄█▇▅▇▃▁▇▆
val/auroc,▁▅▇█▄▂▂▅▇▁
val/f1,▇▂▆█▃▇▁▆▇▄


In [7]:
bert_l.named_parameters()

<generator object Module.named_parameters at 0x7f84298b8940>

In [13]:
norms = {
        f"grad_{2}_norm/{name}": p.grad.data.norm(2.)
        for name, p in bert_l.model.named_parameters()
        if p.grad is not None
    }

In [16]:
for name, p in bert_l.model.named_parameters():
    print(p.grad)

None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None


In [7]:
def overfit_on_single_batch(model):
    data = RottenDataLoader(config, batch_size=4)
    data.setup()
    trainer = Trainer(
        max_epochs=50,
        accelerator="auto",
        devices=1 if torch.cuda.is_available() else None,
        check_val_every_n_epoch=1,
        log_every_n_steps=1,
        overfit_batches=1,
        callbacks=EarlyStopping(monitor="train/loss", mode="min", stopping_threshold=5e-3)
    )
    trainer.fit(model, datamodule=data)
    # assert trainer.logged_metrics["train/loss"] < 5e-3

{'train_loss': tensor(7.5036e-05),
 'val_loss': tensor(1.0620),
 'val/accuracy': tensor(0.6250),
 'val/f1': tensor(0.4000),
 'val/precision': tensor(0.5000),
 'val/recall': tensor(0.3333),
 'val/auroc': tensor(0.7333),
 'train/accuracy': tensor(1.),
 'train/f1': tensor(1.),
 'train/precision': tensor(1.),
 'train/recall': tensor(1.),
 'train/auroc': tensor(1.)}

In [6]:
bert_l.hparams

In [ ]:
import wandb

In [ ]:
train = data.dataset["train"]

In [ ]:
dl = data.train_dataloader()

In [ ]:
for i in dl:
    break

In [ ]:
i.keys()

dict_keys(['input_ids', 'token_type_ids', 'attention_mask', 'labels'])

In [ ]:
next(iter(dl))

ValueError: Unable to create tensor, you should probably activate truncation and/or padding with 'padding=True' 'truncation=True' to have batched tensors with the same length. Perhaps your features (`text` in this case) have excessive nesting (inputs type `list` where type `int` is expected).

In [ ]:
data.tokenizer.decode(data.dataset["train"][0]['input_ids'])

'[CLS] the rock is destined to be the 21st century\'s new " conan " and that he\'s going to make a splash even greater than arnold schwarzenegger, jean - claud van damme or steven segal. [SEP]'

In [ ]:
bert(data.dataset["train"][[0]])

tensor([[-0.2356, -0.0677]], grad_fn=<AddmmBackward0>)

In [ ]:
import wandb

In [ ]:
!pdm run wandb login --anonymously

wandb: Currently logged in as: b-szafranski (tidio-ml). Use `wandb login --relogin` to force relogin


In [ ]:
!wandb login --help

Usage: wandb login [OPTIONS] [KEY]...

  Login to Weights & Biases

Options:
  --cloud        Login to the cloud instead of local
  --host TEXT    Login to a specific instance of W&B
  --relogin      Force relogin if already logged in.
  --anonymously  Log in anonymously
  --verify       Verify login credentials
  --help         Show this message and exit.


In [ ]:
wandb_login_ensure_personal_account()

wandb: Currently logged in as: szafrixxx. Use `wandb login --relogin` to force relogin


In [ ]:
os.environ.get("WANDB_PERSONAL_USERNAME")

'szafrixxx'

In [ ]:
import os

In [ ]:
os.environ["WANDB_API_KEY"] = "0efff4fd3690b7b7a2deb0eb16c619cde3f70b89"

In [ ]:
!wandb login

wandb: Currently logged in as: szafrixxx. Use `wandb login --relogin` to force relogin


In [ ]:
wandb.api.viewer()

{'id': 'VXNlcjoxODA2MjEy',
 'entity': 'szafrixxx',
 'username': 'szafrixxx',
 'flags': '{"name":"default","rate_limit":"400/s","system_metrics":"2/m","sweeps_enabled":false,"teams_enabled":false,"private_projects":true,"gpu_enabled":null,"hub_settings":{"repo":"lukas/ml-class","disk":"10Gi","expiration":259200,"redis_enabled":false,"docker_enabled":false,"image":null},"restricted":false,"proxy_settings":{"openai":null},"noContact":false}',
 'teams': {'edges': [{'node': {'name': 'szafrixxx'}}]}}